In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
data = pd.read_excel("../Dataset.xlsx", sheet_name=['Total Consumers'])
df = data['Total Consumers']
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2.964,2.322959,1.544607,0.778310,1.962012,2.677445,0.237877,0.689194,0.358525,0.814643,...,0.898895,0.203825,0.221624,0.319531,0.830996,0.924987,0.219128,0.274880,0.990488,0.779475
1,2.584,2.371797,1.544607,0.778310,1.962012,2.733737,0.192929,0.558967,0.358525,0.660712,...,0.917793,0.165311,0.179747,0.319531,0.848467,0.944434,0.177722,0.222940,1.011313,0.795863
2,3.071,2.415961,1.319880,0.665072,1.676555,2.784640,0.382869,1.109272,0.377198,1.311186,...,0.934883,0.328060,0.356708,0.336174,0.864266,0.962019,0.352691,0.442426,1.030144,0.810682
3,2.694,2.302538,1.319880,0.665072,1.676555,2.653908,0.442052,1.280743,0.377198,1.513868,...,0.890992,0.378772,0.411848,0.336174,0.823691,0.916855,0.407209,0.510816,0.981781,0.772623
4,2.569,2.363063,0.913154,0.460128,1.159919,2.723669,0.192242,0.556976,0.668500,0.658358,...,0.914413,0.164722,0.179106,0.595793,0.845343,0.940956,0.177089,0.222146,1.007588,0.792932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35131,1.664,2.244719,1.455982,0.733653,1.849437,2.587266,0.205654,0.595835,0.171793,0.704291,...,0.868619,0.176214,0.191602,0.153109,0.803007,0.893832,0.189444,0.237645,0.957128,0.753222
35132,1.659,2.136340,1.201186,0.605264,1.525786,2.462348,0.201219,0.582985,0.067223,0.689101,...,0.826680,0.172414,0.187470,0.059912,0.764237,0.850676,0.185359,0.232519,0.910916,0.716855
35133,1.664,2.192805,1.201186,0.605264,1.525786,2.527430,0.228585,0.662271,0.067223,0.782819,...,0.848530,0.195862,0.212966,0.059912,0.784436,0.873160,0.210568,0.264142,0.934992,0.735802
35134,1.697,1.446083,0.259545,0.130782,0.329682,1.666757,0.189302,0.548459,0.070958,0.648292,...,0.559578,0.162203,0.176368,0.063241,0.517310,0.575820,0.174381,0.218749,0.616596,0.485237


In [103]:
def no_ml_predict(X, y):
    mse = mean_squared_error(X, y)
    mae = mean_absolute_error(X, y)
    r2 = r2_score(X, y)
    print('MSE: %.4f' % mse)
    print('MAE: %.4f' % mae)
    print('R2: %.4f' % r2)

In [113]:
df_public = df[0]

#### Last 15 minutes prediction data

In [114]:
X_last15_public = df_public.copy()
X_last15_public = X_last15_public.iloc[0:(m-1)]
X_last15_public

0        2.964
1        2.584
2        3.071
3        2.694
4        2.569
         ...  
35130    1.670
35131    1.664
35132    1.659
35133    1.664
35134    1.697
Name: 0, Length: 35135, dtype: float64

In [115]:
y_last15_public = df_public.copy()
m = len(X_last15_public)

y_last15_public = y_last15_public.iloc[1:].reset_index(drop=True)
y_last15_public

0        2.584
1        3.071
2        2.694
3        2.569
4        3.174
         ...  
35130    1.664
35131    1.659
35132    1.664
35133    1.697
35134    1.673
Name: 0, Length: 35135, dtype: float64

#### Previous day at same hour prediction data

In [116]:
X_previous_day = df_public.copy()
m = len(X_previous_day)
X_previous_day = X_previous_day.iloc[0:(m-48)]
X_previous_day

0        2.964
1        2.584
2        3.071
3        2.694
4        2.569
         ...  
35083    1.658
35084    1.656
35085    1.679
35086    1.677
35087    1.664
Name: 0, Length: 35088, dtype: float64

In [117]:
y_previous_day = df_public.copy()
y_previous_day = y_previous_day.iloc[48:].reset_index(drop=True)
y_previous_day

0        2.717
1        3.089
2        2.522
3        3.206
4        3.166
         ...  
35083    1.664
35084    1.659
35085    1.664
35086    1.697
35087    1.673
Name: 0, Length: 35088, dtype: float64

#### Previous week same day at same hour prediction data

In [118]:
X_last_week = df_public.copy()
m = len(X_last_week)
X_last_week = X_last_week.iloc[0:(m-48*7)]
X_last_week

0        2.964
1        2.584
2        3.071
3        2.694
4        2.569
         ...  
34795    1.660
34796    1.659
34797    1.664
34798    1.710
34799    1.664
Name: 0, Length: 34800, dtype: float64

In [119]:
y_last_week = df_public.copy()
y_last_week = y_last_week.iloc[(48*7):].reset_index(drop=True)
y_last_week

0        6.596
1        6.653
2        7.054
3        6.929
4        6.717
         ...  
34795    1.664
34796    1.659
34797    1.664
34798    1.697
34799    1.673
Name: 0, Length: 34800, dtype: float64

### Prediction

In [120]:
no_ml_predict(X_last15_public, y_last15_public)

MSE: 0.2680
MAE: 0.3025
R2: 0.9793


In [121]:
no_ml_predict(X_previous_day, y_previous_day)

MSE: 9.0375
MAE: 1.4935
R2: 0.3009


In [122]:
no_ml_predict(X_last_week, y_last_week)

MSE: 28.4673
MAE: 2.0823
R2: -1.1895
